# LoRA-Diffusion Results Analysis

This notebook provides analysis tools for LoRA-Diffusion experiments.

**Contents:**
1. Load and visualize training history
2. Compare methods across tasks
3. Analyze parameter efficiency
4. Visualize step-adaptive ranks
5. Performance vs. efficiency tradeoffs

In [ ]:
# Imports
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("Imports successful!")

## 1. Load Training History

In [ ]:
def load_training_history(output_dir):
    """Load training history from output directory."""
    history_file = Path(output_dir) / "training_history.json"
    
    if not history_file.exists():
        print(f"Warning: {history_file} not found")
        return None
    
    with open(history_file, 'r') as f:
        history = json.load(f)
    
    return pd.DataFrame(history)

# Example: Load a specific experiment
output_dir = "./outputs/sst2_lora_diffusion"  # Change this to your output directory

df = load_training_history(output_dir)

if df is not None:
    print(f"Loaded {len(df)} training steps")
    print("\nColumns:", df.columns.tolist())
    print("\nFirst few rows:")
    display(df.head())
else:
    print("No training history found. Run training first.")

## 2. Visualize Training Curves

In [ ]:
def plot_training_curves(df, metrics=['loss', 'accuracy']):
    """Plot training curves."""
    if df is None:
        print("No data to plot")
        return
    
    fig, axes = plt.subplots(1, len(metrics), figsize=(6*len(metrics), 5))
    
    if len(metrics) == 1:
        axes = [axes]
    
    for ax, metric in zip(axes, metrics):
        # Extract metric values
        if 'metrics' in df.columns:
            values = df['metrics'].apply(lambda x: x.get(metric, np.nan))
        else:
            values = df.get(metric, [])
        
        ax.plot(df['step'], values, linewidth=2)
        ax.set_xlabel('Training Step')
        ax.set_ylabel(metric.replace('_', ' ').title())
        ax.set_title(f'Training {metric.replace("_", " ").title()}')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot training curves
if df is not None:
    plot_training_curves(df, metrics=['loss', 'accuracy'])

## 3. Compare Multiple Methods

In [ ]:
def load_multiple_experiments(base_dir, experiment_names):
    """Load multiple experiment results."""
    results = {}
    
    for name in experiment_names:
        exp_dir = Path(base_dir) / name
        df = load_training_history(exp_dir)
        if df is not None:
            results[name] = df
    
    return results

# Example: Compare different methods on SST-2
experiment_names = [
    "sst2_lora_diffusion",
    "sst2_weight_lora",
    "sst2_full_ft",
    "sst2_adapters",
]

base_dir = "./outputs"
experiments = load_multiple_experiments(base_dir, experiment_names)

print(f"Loaded {len(experiments)} experiments")
for name in experiments:
    print(f"  - {name}: {len(experiments[name])} steps")

In [ ]:
def plot_method_comparison(experiments, metric='loss'):
    """Compare multiple methods."""
    plt.figure(figsize=(12, 6))
    
    for name, df in experiments.items():
        if 'metrics' in df.columns:
            values = df['metrics'].apply(lambda x: x.get(metric, np.nan))
        else:
            values = df.get(metric, [])
        
        # Clean name for legend
        method_name = name.split('_', 1)[1].replace('_', ' ').title()
        plt.plot(df['step'], values, label=method_name, linewidth=2)
    
    plt.xlabel('Training Step')
    plt.ylabel(metric.replace('_', ' ').title())
    plt.title(f'Method Comparison: {metric.replace("_", " ").title()}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot comparison
if experiments:
    plot_method_comparison(experiments, metric='loss')
    plot_method_comparison(experiments, metric='accuracy')

## 4. Parameter Efficiency Analysis

In [ ]:
def analyze_parameter_efficiency(checkpoint_dirs):
    """Analyze parameter efficiency of different methods."""
    results = []
    
    for name, checkpoint_dir in checkpoint_dirs.items():
        config_file = Path(checkpoint_dir) / "config.json"
        
        if not config_file.exists():
            continue
        
        with open(config_file, 'r') as f:
            config = json.load(f)
        
        # Calculate parameter counts (simplified)
        # In practice, you'd load the actual model
        method = config.get('method', {}).get('name', 'unknown')
        
        # Approximate parameter counts
        param_counts = {
            'lora_diffusion': 0.7,
            'weight_lora': 0.9,
            'adapters': 2.1,
            'prefix_tuning': 1.0,
            'bitfit': 0.1,
            'full_ft': 100.0,
        }
        
        results.append({
            'Method': name.split('_', 1)[1].replace('_', ' ').title(),
            'Trainable %': param_counts.get(method, 1.0),
        })
    
    return pd.DataFrame(results)

# Example parameter efficiency data
param_data = pd.DataFrame([
    {'Method': 'LoRA-Diffusion', 'Trainable %': 0.7, 'Performance': 98.1},
    {'Method': 'Weight LoRA', 'Trainable %': 0.9, 'Performance': 94.1},
    {'Method': 'Adapters', 'Trainable %': 2.1, 'Performance': 96.1},
    {'Method': 'Prefix Tuning', 'Trainable %': 1.0, 'Performance': 92.1},
    {'Method': 'BitFit', 'Trainable %': 0.1, 'Performance': 86.5},
    {'Method': 'Full FT', 'Trainable %': 100.0, 'Performance': 100.0},
])

display(param_data)

In [ ]:
def plot_efficiency_frontier(df):
    """Plot performance vs. parameter efficiency."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Scatter plot
    scatter = ax.scatter(
        df['Trainable %'],
        df['Performance'],
        s=200,
        alpha=0.6,
        c=range(len(df)),
        cmap='viridis'
    )
    
    # Add labels
    for _, row in df.iterrows():
        ax.annotate(
            row['Method'],
            (row['Trainable %'], row['Performance']),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=10,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
        )
    
    ax.set_xlabel('Trainable Parameters (%)', fontsize=12)
    ax.set_ylabel('Performance (% of Full FT)', fontsize=12)
    ax.set_title('Parameter Efficiency Frontier', fontsize=14, fontweight='bold')
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_efficiency_frontier(param_data)

## 5. Step-Adaptive Rank Visualization

In [ ]:
def visualize_rank_schedule(num_steps=100):
    """Visualize step-adaptive rank allocation."""
    # Define rank schedule
    steps = np.arange(num_steps)
    ranks = np.zeros(num_steps)
    scalings = np.zeros(num_steps)
    
    # Apply schedule
    for t in range(num_steps):
        if t > 2 * num_steps // 3:  # Early
            ranks[t] = 64
            scalings[t] = 1.0
        elif t > num_steps // 3:  # Mid
            ranks[t] = 32
            scalings[t] = 0.5
        else:  # Late
            ranks[t] = 8
            scalings[t] = 0.25
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Rank
    ax1.plot(steps, ranks, linewidth=3, color='steelblue')
    ax1.set_ylabel('Rank (r)', fontsize=12)
    ax1.set_title('Step-Adaptive Rank Allocation', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0, 70])
    
    # Add phase labels
    ax1.axvspan(67, 100, alpha=0.2, color='red', label='Early (r=64)')
    ax1.axvspan(34, 67, alpha=0.2, color='orange', label='Mid (r=32)')
    ax1.axvspan(0, 34, alpha=0.2, color='green', label='Late (r=8)')
    ax1.legend(loc='upper right')
    
    # Scaling
    ax2.plot(steps, scalings, linewidth=3, color='coral')
    ax2.set_xlabel('Diffusion Step (t)', fontsize=12)
    ax2.set_ylabel('Scaling (σ)', fontsize=12)
    ax2.set_title('Step-Adaptive Scaling', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([0, 1.1])
    
    plt.tight_layout()
    plt.show()

visualize_rank_schedule()

## 6. Load and Compare Final Results

In [ ]:
def load_evaluation_results(result_files):
    """Load evaluation results from multiple experiments."""
    results = []
    
    for name, file_path in result_files.items():
        file_path = Path(file_path)
        if not file_path.exists():
            continue
        
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        metrics = data.get('metrics', {})
        metrics['Method'] = name
        results.append(metrics)
    
    return pd.DataFrame(results)

# Example: Load results (update paths as needed)
result_files = {
    'LoRA-Diffusion': './outputs/sst2_lora_diffusion/eval_results.json',
    'Weight LoRA': './outputs/sst2_weight_lora/eval_results.json',
    'Full FT': './outputs/sst2_full_ft/eval_results.json',
}

# results_df = load_evaluation_results(result_files)
# display(results_df)

print("Note: Update result_files paths with your actual output directories")

## 7. Multi-Task Comparison

In [ ]:
def create_comparison_heatmap(data):
    """Create heatmap comparing methods across tasks."""
    plt.figure(figsize=(10, 6))
    
    sns.heatmap(
        data,
        annot=True,
        fmt='.1f',
        cmap='RdYlGn',
        center=80,
        vmin=70,
        vmax=95,
        cbar_kws={'label': 'Performance Score'}
    )
    
    plt.title('Method Performance Across Tasks', fontsize=14, fontweight='bold')
    plt.xlabel('Task', fontsize=12)
    plt.ylabel('Method', fontsize=12)
    plt.tight_layout()
    plt.show()

# Example data (from paper Table 3)
comparison_data = pd.DataFrame({
    'SST-2': [94.2, 92.1, 93.4, 91.8, 94.5],
    'SQuAD': [87.9, 84.3, 86.9, 83.1, 88.7],
    'XSum': [37.4, 35.2, 36.9, 34.3, 37.8],
}, index=['LoRA-Diffusion', 'Weight LoRA', 'Adapters', 'Prefix', 'Full FT'])

create_comparison_heatmap(comparison_data)

## 8. Training Time Analysis

In [ ]:
def plot_training_time_comparison(data):
    """Compare training times."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = plt.cm.viridis(np.linspace(0, 0.9, len(data)))
    bars = ax.barh(data['Method'], data['Time (hours)'], color=colors)
    
    ax.set_xlabel('Training Time (hours)', fontsize=12)
    ax.set_title('Training Time Comparison (SST-2, 5000 steps)', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, (bar, time) in enumerate(zip(bars, data['Time (hours)'])):
        ax.text(time + 0.02, bar.get_y() + bar.get_height()/2, 
                f'{time:.2f}h', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Example data
time_data = pd.DataFrame([
    {'Method': 'LoRA-Diffusion', 'Time (hours)': 0.45},
    {'Method': 'Weight LoRA', 'Time (hours)': 0.54},
    {'Method': 'Adapters', 'Time (hours)': 0.72},
    {'Method': 'Full FT', 'Time (hours)': 0.91},
])

plot_training_time_comparison(time_data)

## 9. Summary Statistics

In [ ]:
def print_summary_statistics(experiments):
    """Print summary statistics for experiments."""
    print("=" * 80)
    print("EXPERIMENT SUMMARY")
    print("=" * 80)
    
    for name, df in experiments.items():
        print(f"\n{name}:")
        print("-" * 40)
        
        # Get final metrics
        if 'metrics' in df.columns and len(df) > 0:
            final_metrics = df.iloc[-1]['metrics']
            print(f"  Final Loss: {final_metrics.get('loss', 'N/A'):.4f}")
            print(f"  Final Accuracy: {final_metrics.get('accuracy', 'N/A'):.4f}")
        
        # Training time
        if 'time_per_step' in df.columns:
            total_time = df['time_per_step'].sum() / 3600  # Convert to hours
            print(f"  Total Training Time: {total_time:.2f} hours")
    
    print("\n" + "=" * 80)

if experiments:
    print_summary_statistics(experiments)

## 10. Export Results for Paper

In [ ]:
def export_latex_table(df, caption="Results comparison", label="tab:results"):
    """Export results as LaTeX table."""
    latex = df.to_latex(
        index=True,
        float_format="%.2f",
        caption=caption,
        label=label,
        escape=False,
    )
    
    print("LaTeX Table:")
    print("=" * 80)
    print(latex)
    print("=" * 80)
    
    # Save to file
    with open('results_table.tex', 'w') as f:
        f.write(latex)
    print("\nSaved to results_table.tex")

# Example
# export_latex_table(comparison_data, caption="Performance across tasks", label="tab:performance")

## Conclusion

This notebook provides tools for analyzing LoRA-Diffusion results. Key analyses:

1. **Training curves**: Monitor loss and accuracy over time
2. **Method comparison**: Compare different PEFT methods
3. **Parameter efficiency**: Analyze trainable parameters vs. performance
4. **Step-adaptive ranks**: Visualize rank allocation strategy
5. **Multi-task results**: Compare performance across tasks
6. **Training time**: Analyze computational efficiency

**Next steps:**
- Update paths to your actual experiment directories
- Run your own analyses
- Export results for papers/presentations
- Customize visualizations as needed